In [1]:
import os
import random
root = "/content/drive/MyDrive/dataset"
gen = os.walk(root)
list_of_folders = next(gen)[1]
list_of_folders.sort()

train = []
valid = []

for folder in list_of_folders:
  r1 = os.path.join(root, folder)
  g = os.walk(r1)
  
  for subfolder in next(g)[1]:
    r2 = (os.path.join(r1, subfolder))

    files = [f for f in os.listdir(r2) if os.path.isfile(os.path.join(r2, f))]

    for file_name in files:
      if random.uniform(0, 1) < 0.8:
        train.append([os.path.join(r2, file_name), 1 if subfolder == "fall" else 0])
      else:
        valid.append([os.path.join(r2, file_name), 1 if subfolder == "fall" else 0])

# 80 - 20 split to traning & validation data

In [2]:
import pandas as pd

train_df = pd.DataFrame(train, columns=['path', 'fall'])
valid_df = pd.DataFrame(valid, columns=['path', 'fall'])
print(train_df)

                                                 path  fall
0   /content/drive/MyDrive/dataset/Anju/notFall/da...     0
1   /content/drive/MyDrive/dataset/Anju/notFall/da...     0
2   /content/drive/MyDrive/dataset/Anju/notFall/da...     0
3   /content/drive/MyDrive/dataset/Anju/notFall/da...     0
4   /content/drive/MyDrive/dataset/Anju/notFall/da...     0
..                                                ...   ...
87  /content/drive/MyDrive/dataset/dhirendraBhai/n...     0
88  /content/drive/MyDrive/dataset/dhirendraBhai/n...     0
89  /content/drive/MyDrive/dataset/dhirendraBhai/n...     0
90  /content/drive/MyDrive/dataset/dhirendraBhai/n...     0
91  /content/drive/MyDrive/dataset/dhirendraBhai/n...     0

[92 rows x 2 columns]


In [3]:
train_df.fall.value_counts()

0    48
1    44
Name: fall, dtype: int64

In [4]:
valid_df.fall.value_counts()

1    15
0    11
Name: fall, dtype: int64

In [5]:
shapes = []
import torch

for i in range(len(train_df)):
  csv_path = train_df.iloc[i, 0]
  mat = pd.read_csv(csv_path)
  mat = [list(mat[x]) for x in ['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
  mat = torch.tensor(mat)
  if mat.shape[1] < 10:
    print(csv_path)
  shapes.append(mat.shape[1])

print(shapes)
mat_min = min(shapes) # csv to tensor

[126, 121, 120, 124, 121, 120, 123, 119, 122, 127, 127, 128, 129, 134, 127, 132, 138, 134, 132, 138, 139, 136, 136, 133, 137, 134, 136, 134, 134, 136, 135, 136, 135, 128, 130, 129, 132, 143, 137, 141, 142, 138, 135, 142, 121, 121, 123, 94, 120, 128, 122, 119, 128, 130, 126, 128, 128, 133, 131, 88, 134, 133, 135, 142, 134, 137, 134, 121, 129, 133, 132, 134, 134, 135, 133, 133, 136, 127, 129, 128, 130, 129, 129, 131, 132, 136, 137, 138, 138, 139, 136, 138]


In [6]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset, IterableDataset

def mat_to_tensor(mat_path):
  mat = pd.read_csv(mat_path)
  mat = mat.dropna()
  mat = [list(mat[x]) for x in ['ax', 'ay', 'az', 'gx', 'gy', 'gz']]
  mat = torch.tensor(mat)
  return mat

class IMUSet(Dataset):
    def __init__(self, data_df):
        self.data_df = data_df

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        mat_path = self.data_df.iloc[idx, 0]
        t = mat_to_tensor(mat_path)
        label = self.data_df.iloc[idx, 1]
        return t, label

In [7]:
from torch.utils.data import DataLoader

imuset = IMUSet(train_df) # make dataset

train_dataloader = DataLoader(imuset, shuffle=True) 

In [8]:
valid_dataset = IMUSet(valid_df)
valid_dataloader = DataLoader(valid_dataset, shuffle=True)

valid_features, valid_labels = next(iter(valid_dataloader))

valid_features.shape, valid_labels.shape

(torch.Size([1, 6, 133]), torch.Size([1]))

**LSTM**

In [9]:
from torch import nn
import torch.nn.functional as F

class FallDetector(nn.Module):
    def __init__(self):
        super(FallDetector, self).__init__()
        self.lstm = nn.LSTM(input_size=6, hidden_size=10, num_layers=1)
        self.hidden2tag = nn.Linear(10, 1)

    def forward(self, seq):
        output, (h_n, c_n) = self.lstm(seq.view(len(seq), 1, -1))
        tag_space = self.hidden2tag(c_n.view(1, -1))
        # print(tag_space.data)
        tag_scores = torch.sigmoid(tag_space)
        return tag_scores.view(-1)

In [10]:
import torch.optim as optim

model = FallDetector()
loss_function = nn.BCELoss() 
optimizer = optim.Adam(model.parameters())

In [11]:
import numpy as np

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

def get_statistics(y_true, y_pred):
    y_pred = np.concatenate(tuple(y_pred)) > 0.5
    y_true = np.concatenate(tuple([[t for t in y] for y in y_true])).reshape(y_pred.shape)

    print("Accuracy: ", accuracy_score(y_true, y_pred))
    print("Recall: ", recall_score(y_true, y_pred))
    print(len(y_true))

    print("Confusion Matrix")
    print(confusion_matrix(y_true, y_pred))

In [12]:
for epoch in range(50): 
    labels = []
    preds = []
    loss = []

    i = 0
    for seq, label in train_dataloader:
        model.zero_grad()

        seq = seq.reshape((-1, 6))
        tag_scores = model(seq.to(torch.float))
        preds.append(tag_scores.data)
        labels.append(label.to(torch.float).data)

        loss = loss_function(tag_scores, label.to(torch.float))
        loss.backward()
        optimizer.step()

        i += 1

    print("Epoch Done")
    get_statistics(labels, preds)

    # tag_scores[tag_scores > 0.4] = 1


Epoch Done
Accuracy:  0.6304347826086957
Recall:  0.6363636363636364
92
Confusion Matrix
[[30 18]
 [16 28]]
Epoch Done
Accuracy:  0.7282608695652174
Recall:  0.8181818181818182
92
Confusion Matrix
[[31 17]
 [ 8 36]]
Epoch Done
Accuracy:  0.7065217391304348
Recall:  0.8636363636363636
92
Confusion Matrix
[[27 21]
 [ 6 38]]
Epoch Done
Accuracy:  0.7282608695652174
Recall:  0.8409090909090909
92
Confusion Matrix
[[30 18]
 [ 7 37]]
Epoch Done
Accuracy:  0.7282608695652174
Recall:  0.7272727272727273
92
Confusion Matrix
[[35 13]
 [12 32]]
Epoch Done
Accuracy:  0.7608695652173914
Recall:  0.8636363636363636
92
Confusion Matrix
[[32 16]
 [ 6 38]]
Epoch Done
Accuracy:  0.75
Recall:  0.8863636363636364
92
Confusion Matrix
[[30 18]
 [ 5 39]]
Epoch Done
Accuracy:  0.6739130434782609
Recall:  0.5909090909090909
92
Confusion Matrix
[[36 12]
 [18 26]]
Epoch Done
Accuracy:  0.6304347826086957
Recall:  0.5454545454545454
92
Confusion Matrix
[[34 14]
 [20 24]]
Epoch Done
Accuracy:  0.6413043478260869
R

In [16]:
labels = []
preds = []
loss = []
model.eval()

for seq, label in valid_dataloader:

    seq = seq.reshape((-1, 6))
    tag_scores = model(seq.to(torch.float))
    preds.append(tag_scores.data)
    labels.append(label.to(torch.float).data)

print("Epoch Done")
get_statistics(labels, preds)

Epoch Done
Accuracy:  0.7307692307692307
Recall:  0.6666666666666666
26
Confusion Matrix
[[ 9  2]
 [ 5 10]]


In [14]:
# torch.save(model.state_dict(), 'model_weights.pt')

In [15]:
# from torch import nn
# import torch.nn.functional as F

# class FallDetector(nn.Module):
#     def __init__(self):
#         super(FallDetector, self).__init__()
#         self.lstm = nn.LSTM(6, 5, num_layers=1)
#         self.hidden2tag = nn.Linear(5, 1)

#     def forward(self, seq):
#         output, (h_n, c_n) = self.lstm(seq.view(len(seq), 1, -1))
#         tag_space = self.hidden2tag(c_n.view(1, -1))
#         # print(tag_space.data)
#         tag_scores = torch.sigmoid(tag_space)
#         return tag_scores.view(-1)

# model = FallDetector()
# model.load_state_dict(torch.load('model_weights.pt'))
# model.eval()